In [ ]:
# Step 1: Import Required Libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

# Step 2: API Configuration
API_KEY = "dfcd16d8f5058144250d9e2a9279fccd"  # Replace with your Last.fm API key
USER = "the_atm"  # Replace with your Last.fm username
BASE_URL = "http://ws.audioscrobbler.com/2.0/"  # Base URL for Last.fm API

# Step 3: Fetch Data from Last.fm API
def fetch_lastfm_data(user, api_key):
    # Fetch the user's recent tracks
    response = requests.get(BASE_URL, {
        "method": "user.getrecenttracks",
        "user": user,
        "api_key": api_key,
        "format": "json",
        "limit": 1000  # Adjust limit as needed
    })
    data = response.json()
    return data

# Call the API
print("Fetching data from Last.fm...")
data = fetch_lastfm_data(USER, API_KEY)

# Step 4: Extract and Preprocess Data
# Parse the JSON data into a DataFrame
tracks = data['recenttracks']['track']
df = pd.DataFrame([{
    "user_id": 0,  # Assign numeric ID for the single user
    "song_name": track['name'],
    "artist_name": track['artist']['#text']
} for track in tracks])

print("Sample Data:")
print(df.head())

# Encode song and artist names into numeric IDs
df['song_id'] = LabelEncoder().fit_transform(df['song_name'])
df['artist_id'] = LabelEncoder().fit_transform(df['artist_name'])

# Add interaction column (e.g., play count or implicit rating)
df['interaction'] = 1  # Treat all plays equally

# Split into train and test sets
train, test = train_test_split(df, test_size=0.2, random_state=42)

# Step 5: Build the Deep Learning Model
# Define parameters
num_users = 1  # Only one user (USER)
num_songs = df['song_id'].nunique()
embedding_dim = 50

# Input layers
user_input = tf.keras.layers.Input(shape=(1,))
song_input = tf.keras.layers.Input(shape=(1,))

# Embedding layers
user_embedding = tf.keras.layers.Embedding(num_users, embedding_dim)(user_input)
song_embedding = tf.keras.layers.Embedding(num_songs, embedding_dim)(song_input)

# Flatten embeddings
user_vec = tf.keras.layers.Flatten()(user_embedding)
song_vec = tf.keras.layers.Flatten()(song_embedding)

# Concatenate embeddings
concat = tf.keras.layers.Concatenate()([user_vec, song_vec])
dense = tf.keras.layers.Dense(128, activation='relu')(concat)
output = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

# Create model
model = tf.keras.Model(inputs=[user_input, song_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Step 6: Train the Model
# Prepare input data
X_train = [train['user_id'], train['song_id']]
y_train = train['interaction']

X_test = [test['user_id'], test['song_id']]
y_test = test['interaction']

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Step 7: Evaluate the Model
results = model.evaluate(X_test, y_test)
print(f"Test Loss: {results[0]}, Test Accuracy: {results[1]}")

# Step 8: Test Recommendations for a User
# Generate song recommendations
def recommend_songs(num_recommendations=5):
    user_array = np.array([0] * num_songs)  # Single user
    song_array = np.array(range(num_songs))

    predictions = model.predict([user_array, song_array])
    recommended_songs = predictions.flatten().argsort()[-num_recommendations:][::-1]

    recommended_df = df[df['song_id'].isin(recommended_songs)]
    print("\nRecommended Songs:")
    print(recommended_df[['song_name', 'artist_name']].drop_duplicates())
    return recommended_df[['song_name', 'artist_name']].drop_duplicates()

# Get recommendations
recommendations = recommend_songs()


Fetching data from Last.fm...
Sample Data:
   user_id                          song_name artist_name
0        0                 This Is Not A Song     Siamese
1        0                     God Is A Woman     Siamese
2        0  The Shape of Water (feat. ten56.)     Siamese
3        0               Memories On The Wind  Fellowship
4        0                   The Bitter Winds  Fellowship


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12            │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_13            │ (None, 1)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_12 (Embedding)  │ (None, 1, 50)          │             50 │ input_layer_12[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_13 (Embedding)  │ (None, 1, 50)          │         40,050 │ input_layer_13[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_12 (Flatten)      │ (None, 50)             │              0 │ embedding_12[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_13 (Flatten)      │ (None, 50)             │              0 │ embedding_13[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_6             │ (None, 100)            │              0 │ flatten_12[0][0],      │
│ (Concatenate)             │                        │                │ flatten_13[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 128)            │         12,928 │ concatenate_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_13 (Dense)          │ (None, 1)              │            129 │ dense_12[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 53,157 (207.64 KB)

 Trainable params: 53,157 (207.64 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8970 - loss: 0.5969 - val_accuracy: 1.0000 - val_loss: 0.1926
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.1053 - val_accuracy: 1.0000 - val_loss: 0.0075
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0049 - val_accuracy: 1.0000 - val_loss: 0.0020
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0016 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 1.0000 - val_loss: 8.8836e-04
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 7.7571e-04 - val_accuracy: 1.0000 - val_loss: 6.7025e-04
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 5.8874e-04 - val_accuracy: 1.0000 - val_loss: 5.1916e-04
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 4.5696e-04 - val_acc